In [1]:
# update rmse 並記得重新啟動 Anaconda
# !pip install -U scikit-learn 

In [14]:
# Modules
import random
import pandas as pd
import numpy as np
from keras import models
from keras import layers
from keras import optimizers, metrics
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.layers.advanced_activations import LeakyReLU
import math
import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats
from scipy.stats import norm, skew #for some statistics
from sklearn.preprocessing import PowerTransformer

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler

from sklearn.metrics import r2_score
from keras import backend as K

import warnings
warnings.filterwarnings('ignore')

In [15]:

def lr_rmse_ave(x,y):
    
    
    train_rmse=[]
    test_rmse=[]
    test_r2=[]
    
    for i in np.arange(10):
        x_train,x_test,y_train,y_test = train_test_split(x,y, random_state = i)
        lr = LinearRegression().fit(x_train,y_train)

        y_train_pred = lr.predict(x_train)
        y_test_pred = lr.predict(x_test)
        train_rmse.append(mean_squared_error(y_train, y_train_pred,squared=False))
        test_rmse.append(mean_squared_error(y_test, y_test_pred,squared=False))
        test_r2.append(r2_score(y_test,y_test_pred))
    
    train_rmse=np.array(train_rmse).mean()
    test_rmse=np.array(test_rmse).mean()
    test_r2=np.array(test_r2).mean()
    
#     print('train_rmse:', train_rmse)
#     print('test_rmse:', test_rmse)
#     print('test_r2:', test_r2)
    return train_rmse, test_rmse, test_r2

In [16]:
def skew_pro(data):
    columns=data.drop(['Predict'], axis=1).columns
    for col in columns: 
        if abs(data[col].skew()) >= 0.7: 
            pt = PowerTransformer() 
            d = pt.fit_transform(data[col].values.reshape(-1, 1)).flatten()
            data[col]=d
    X = data[columns]
    scaler = RobustScaler()
    data[columns] = scaler.fit_transform(X)
    
    return data

In [17]:
def r2_keras(y_true, y_pred):
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [18]:
def nn_ave(x,y):
    
    rmse=[]
    r2=[]

    for i in np.arange(10):
        X_train,X_val,Y_train,Y_val  = train_test_split(x, y, test_size=0.2, random_state=i)
        
        os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
        batch_size = 32
        epochs = 1000
        model = build_model()
        history = model.fit(
            X_train,
            Y_train,
            epochs = epochs,
            validation_data = (X_val, Y_val),
            batch_size = batch_size,
            shuffle = True,
            verbose=0
            )
        
        y_pred = model.predict(X_val)
        rmse.append(math.sqrt(mean_squared_error(Y_val , y_pred )))
        r2.append(r2_score(Y_val , y_pred ))
    
    test_rmse=np.array(rmse).mean()
    print('test_rmse_ave:',test_rmse)
    print(rmse)
    print('\n')
    test_r2=np.array(r2).mean()
    print('test_r2_ave:',test_r2)
    print(r2)

In [7]:
## 
n='Input_C_046' # 改名稱
##
dire= ['Data/BayesianRidge_Pre_0/','Data/BayesianRidge_Pre_1/','Data/BayesianRidge_Pre_2/','Data/BayesianRidge_Pre_3/','Data/BayesianRidge_Pre_4/','Data/BayesianRidge_Pre_5/','Data/BayesianRidge_Pre_6/','Data/BayesianRidge_Pre_7/']
print(n)
for d in dire:
    f_train_rmse=[]
    f_test_rmse=[]
    f_test_r2=[]
    print(d)
    data_d= d+n+'.csv'
    data= pd.read_csv(data_d,index_col=0).drop(['Number'], axis=1)
    y=data.Predict
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    f20=abs(numcorr['Predict']).sort_values(ascending=False).head(21).to_frame().index.to_numpy()[1:]
    fea_list=[2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
    for i in fea_list:
        print('feature number:',i)
        x=skew_data[f20[:i]]
        tr_r,te_r, te_r2= lr_rmse_ave(x,y)
        print('train_rmse:',tr_r)
        print('test_rmse:',te_r)
        print('test_r2:',te_r2)
        f_train_rmse.append(tr_r)
        f_test_rmse.append(te_r)
        f_test_r2.append(te_r2)

    test_r=np.array(f_test_rmse)
    test_r2=np.array(f_test_r2)
    test_r_min= np.where(test_r == np.amin(test_r))
    test_r2_max= np.where(test_r2 == np.amax(test_r2))    
    print('Test_rmse_min = 1.fea_num:', fea_list[test_r_min[0][0]], ' 2.rmse:' ,np.amin(test_r))
    print('Test_r2_max = 1. fea_num:', fea_list[test_r2_max[0][0]],  ' 2.r2:', np.amax(test_r2))

Input_C_046
Data/BayesianRidge_Pre_0/
feature number: 2
train_rmse: 0.00026177193898245
test_rmse: 0.00026738163531187165
test_r2: 0.35924764902394346
feature number: 3
train_rmse: 0.0002600119037693484
test_rmse: 0.00026589776203324604
test_r2: 0.3664667769128975
feature number: 4
train_rmse: 0.0002575090732699339
test_rmse: 0.00026436883235909984
test_r2: 0.3738470141987322
feature number: 5
train_rmse: 0.00025744315277374775
test_rmse: 0.0002647824603793598
test_r2: 0.37181537338662524
feature number: 6
train_rmse: 0.0002571925977184348
test_rmse: 0.0002657490748369225
test_r2: 0.36707070545860304
feature number: 7
train_rmse: 0.0002557071537721854
test_rmse: 0.00026642122201910344
test_r2: 0.36357307864727095
feature number: 8
train_rmse: 0.00025453600987922323
test_rmse: 0.00026489187843067326
test_r2: 0.37072860527202506
feature number: 9
train_rmse: 0.0002542645551308157
test_rmse: 0.00026640161953553874
test_r2: 0.36328693346155216
feature number: 10
train_rmse: 0.0002530713558

train_rmse: 0.00025015720595935486
test_rmse: 0.00026696213280503556
test_r2: 0.36098365918413317
feature number: 14
train_rmse: 0.00024658139939489964
test_rmse: 0.0002652150423739973
test_r2: 0.3691994597442083
feature number: 15
train_rmse: 0.0002462734038356658
test_rmse: 0.00026539189404018533
test_r2: 0.36835484403594376
feature number: 16
train_rmse: 0.00024275849200479713
test_rmse: 0.00025912965313323026
test_r2: 0.39820284323331545
feature number: 17
train_rmse: 0.00024223171506459752
test_rmse: 0.00026001076721970104
test_r2: 0.3939210252824906
feature number: 18
train_rmse: 0.00024193394550895068
test_rmse: 0.000260939556442432
test_r2: 0.3895197602605426
feature number: 19
train_rmse: 0.00024163552541786454
test_rmse: 0.00026186973071607936
test_r2: 0.38477978812240476
feature number: 20
train_rmse: 0.00023983913368330235
test_rmse: 0.0002618969649253059
test_r2: 0.384908626730675
Test_rmse_min = 1.fea_num: 16  2.rmse: 0.00025912965313323026
Test_r2_max = 1. fea_num: 16  2

train_rmse: 0.00025744315277374775
test_rmse: 0.0002647824603793598
test_r2: 0.37181537338662524
feature number: 6
train_rmse: 0.0002571925977184348
test_rmse: 0.0002657490748369225
test_r2: 0.36707070545860304
feature number: 7
train_rmse: 0.0002557071537721854
test_rmse: 0.00026642122201910344
test_r2: 0.36357307864727095
feature number: 8
train_rmse: 0.00025452169639462953
test_rmse: 0.0002648734253315512
test_r2: 0.3708186620152117
feature number: 9
train_rmse: 0.00025425027892647575
test_rmse: 0.0002663831998780331
test_r2: 0.36337680218164997
feature number: 10
train_rmse: 0.0002530591765609679
test_rmse: 0.00026733027252517363
test_r2: 0.3588939218914261
feature number: 11
train_rmse: 0.00025188027693890324
test_rmse: 0.00026683333158104903
test_r2: 0.36184094617820883
feature number: 12
train_rmse: 0.00025133537734514937
test_rmse: 0.00026707876310481623
test_r2: 0.3604191367653391
feature number: 13
train_rmse: 0.00025011191695842593
test_rmse: 0.0002669212620937017
test_r2: 0

# 可以修改第一層的神經元數量

In [19]:
#
def build_model():
    model = models.Sequential()
#
    model.add(layers.Dense(4, activation='relu',input_shape=(x.shape[1],))) # 調這裡layers的數量
#
    model.add(layers.Dense(1, activation='linear'))
    model.compile(optimizer='adam', loss=metrics.mean_squared_error, metrics=[r2_keras])

    return model

In [21]:
fea_num=[4,5,6,7] # 要測試的feature數量 神經元4

for fea in fea_num:
    
    print('feature number:',fea)
    #
    data=pd.read_csv('Data/BayesianRidge_Pre_4/Input_C_046.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑!!!
    #
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea+1).to_frame().index.to_numpy()[1:]
    x = skew_data[title]
    y = skew_data.Predict
    nn_ave(x,y)
    print('\n')

feature number: 4
test_rmse_ave: 0.005967518007582055
[0.0040394486781679595, 0.037252537661446296, 0.0003068357981822422, 0.004914187739619914, 0.0002707246425645621, 0.002913169263587103, 0.0030608926294916033, 0.00030931129841833995, 0.006349977303555881, 0.00025809506078665215]


test_r2_ave: -1216.4824999269017
[-132.55900408096696, -11309.40329313441, 0.21789570465758934, -196.48288744183833, 0.2984107837609963, -86.66813293852488, -79.28168932839557, 0.2945836661610166, -361.51614579796023, 0.2752632985005963]


feature number: 5
test_rmse_ave: 0.0038133346369747813
[0.012676239933638107, 0.010729056717163347, 0.00033250838557595666, 0.0017642932391844265, 0.0004512844618125655, 0.0002949685240701036, 0.0011226578083275308, 0.0004968420033000505, 0.00977029370421527, 0.0004952015924604554]


test_r2_ave: -314.6170392807108
[-1314.2547660307114, -937.1870804727369, 0.08154512865835295, -24.45467287473988, -0.9495217655604153, 0.10120288547004508, -9.799785105728317, -0.8200803117